In [ ]:
import apache_log_parser
from rich.progress import track
import pandas as pd

line_parser = apache_log_parser.make_parser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")

log_file = open('data/access.log', 'r')
log = []

log = [line for line in log_file]

log_file.close()

parsed_log = [line_parser(line) for line in track(log)]

In [ ]:
df = pd.DataFrame(parsed_log)

small_df = df[['remote_host', 'request_url_path', 'time_received_datetimeobj', 'response_bytes_clf']].dropna()
small_df['response_bytes_clf'] = pd.to_numeric(small_df['response_bytes_clf'])

In [ ]:
filtered_small_df = small_df[~small_df['request_url_path'].str.contains('\.')]
endpoint_list = ('/trail_rankings', '/rankings', '/search', '/about', '/map', '/interactive-map', '/explore')
filtered_small_df = filtered_small_df[filtered_small_df['request_url_path'].str.startswith(endpoint_list)]

In [ ]:
filtered_small_df.describe()

In [ ]:
filtered_small_df.remote_host.value_counts().head(50)

In [ ]:
filtered_small_df['date'] = pd.to_datetime(filtered_small_df['time_received_datetimeobj']).dt.date

In [ ]:
spam_df = filtered_small_df[filtered_small_df['remote_host'] == '73.17.249.14']
spam_df.head(10)

In [ ]:
trimmed_df = filtered_small_df[filtered_small_df['date'] > pd.to_datetime('2023-03-01').date()]

In [ ]:
trimmed_df['date'].hist()

In [ ]:
trimmed_df[['response_bytes_clf', 'date']].groupby(['date']).sum().plot()